pip install openpyxl   
pip install xlrd

In [1]:
import numpy as np
import pandas as pd
import requests,json,os
from urllib.parse import quote 

In [2]:
df = pd.read_excel('../../전체_도시철도역사정보_20230317.xlsx',engine='openpyxl')
df.head()

,역번호,역사명,노선번호,노선명,영문역사명,한자역사명,환승역구분,환승노선번호,환승노선명,역위도,역경도,운영기관명,역사도로명주소,역사전화번호,데이터기준일자
0,0101,판암,S3001,대전 도시철도 1호선,Panam,板岩,일반역,NaN,NaN,36.316896,127.458264,대전교통공사,대전광역시 동구 옥천로 지하162 (판암동),042-539-3601,2022-05-09 00:00:00
1,0102,신흥,S3001,대전 도시철도 1호선,Sinheung,新興,일반역,NaN,NaN,36.319652,127.448792,대전교통공사,대전광역시 동구 옥천로 지하 70 (판암동),042-539-3602,2022-05-09 00:00:00
2,0103,대동,S3001,대전 도시철도 1호선,Dae-dong,大洞,일반역,NaN,NaN,36.329532,127.442848,대전교통공사,대전광역시 동구 계족로 지하 176 (대동),042-539-3603,2022-05-09 00:00:00
3,0104,대전,S3001,대전 도시철도 1호선,Daejeon,大田,일반역,NaN,NaN,36.331583,127.433118,대전교통공사,대전광역시 동구 중앙로 지하 218 (중동),042-539-3604,2022-05-09 00:00:00
4,0105,중앙로,S3001,대전 도시철도 1호선,Jungangno,中央路,일반역,NaN,NaN,36.328664,127.425842,대전교통공사,대전광역시 중구 중앙로 지하 145 (은행동),042-539-3605,2022-05-09 00:00:00


In [3]:
df.to_csv('./total_info.csv',index=False)

In [4]:
df = pd.read_csv('./total_info.csv')
df = df[['역사명','역사도로명주소','운영기관명']]
df.rename(columns={'역사명':'지하철역','역사도로명주소':'도로명주소'},inplace=True)
df.운영기관명.unique()
exclude_list = ['대전교통공사', '대구도시철도공사', '부산광역시 부산교통공사', '부산-김해경전철㈜','광주광역시 도시철도공사']
for exclude in exclude_list:
    df = df[df['운영기관명'] != exclude]
df.head()
df.drop(columns=['운영기관명'],inplace=True)
df.shape

(782, 2)

In [5]:
df.to_csv('test_local_info.csv',index=False)

In [6]:
df.reset_index(drop=True,inplace=True)
df.nunique()

지하철역     702
도로명주소    738
dtype: int64

In [7]:
df.head()

,지하철역,도로명주소
0,계양,인천광역시 계양구 다남로 24
1,귤현,인천광역시 계양구 장제로 1136
2,박촌,인천광역시 계양구 장제로 992
3,임학,인천광역시 계양구 장제로 875
4,계산,인천광역시 계양구 경명대로 1089


In [8]:
df['지하철역'][1][:-1]

'귤'

In [9]:
df['지하철역'] = df['지하철역'].str.replace('(', ' ',regex=False,).str.split().str[0]
for i in df.index:
    if df['지하철역'][i][-1] == '역':
        df['지하철역'][i] = df['지하철역'][i][:-1]
df.drop_duplicates(subset=['지하철역'],keep='first',inplace=True)
df.head()

,지하철역,도로명주소
0,계양,인천광역시 계양구 다남로 24
1,귤현,인천광역시 계양구 장제로 1136
2,박촌,인천광역시 계양구 장제로 992
3,임학,인천광역시 계양구 장제로 875
4,계산,인천광역시 계양구 경명대로 1089


In [10]:
df.nunique()

지하철역     656
도로명주소    648
dtype: int64

In [21]:
####################################################################
# 위까지는 전처리
# 아래부터는 도로명주소 기반으로 좌표 구해서 추가함
####################################################################

In [11]:
df.to_csv('test_local_info.csv',index=False)
# 하고 나서 영종 -> 강조문자 삭제
# 도로명주소에 오타, 잘못된 주소 기입된 역명들 수정
# local_info.csv가 수정본

In [12]:
# 1
# 수정본 파일로 데이터프레임 만듬
df = pd.read_csv('test_local_info.csv')

In [13]:
df.head()

,지하철역,도로명주소
0,계양,인천광역시 계양구 다남로 24
1,귤현,인천광역시 계양구 장제로 1136
2,박촌,인천광역시 계양구 장제로 992
3,임학,인천광역시 계양구 장제로 875
4,계산,인천광역시 계양구 경명대로 1089


In [14]:
# 2 kakao API로 도로명 주소 구하는 함수
def kakao_location(place):
    with open('data/main/key/kakaoapikey.txt') as f_:
        kakao_key = f_.read()
    base_url = "https://dapi.kakao.com/v2/local/search/address.json"
    url = f'{base_url}?query={quote(place)}'
    header = {'Authorization':f'KakaoAK {kakao_key}'}
    result = requests.get(url, headers=header).json()
    lat_ = float(result['documents'][0]['y'])
    lng_ = float(result['documents'][0]['x'])
    return lat_,lng_

In [15]:
# 2 입력받은 데이터 프레임의 인덱스로 도로명주소 추출하는 함수
def rtn_addr(target):
    str_addr = df[df.지하철역 == target].도로명주소.values[-1]
    return str_addr.strip()

In [16]:
# 3 2번 함수들 사용해서 좌표추가
temp1 =[]
for i in df.index:
    try:
        target = df['지하철역'][i].strip()
        temp1.append(kakao_location(rtn_addr(target)))
    except:
        print(i, df.지하철역[i])
        
df_test = pd.DataFrame(temp1,columns=('lat','lng'))
df2 = pd.concat([df, df_test], axis=1)

6 작전
86 청라국제도시
102 성신여대입구
430 성환
437 탕정
470 덕계
491 야당
499 덕소
502 운길산
504 신원
506 아신
508 원덕
511 금릉
540 사릉
544 마석
554 춘천
565 오시리아
574 개운포
581 개포동
582 대모산입구
596 망포
601 오목천
602 어천
604 사리
626 대공원
639 원당
640 화정
653 선부
655 원시


In [18]:
# 4 처리 끝난 파일 저장
df2.to_csv('stn_r_addr_final.csv',index=False)

In [125]:
# 위 과정에서 에러 발생한 부분 검증
temp = df['지하철역'][430]
temp

'성환'

In [126]:
# 결과적으론 이 과정에서는 에러 발생안함
rtn_addr(temp)

'충청남도 천안시 서북구 성환읍 성환1로 237-5 (성환리 449-490번지)'

In [48]:
# 공공데이터에서 제공한 데이터들에 오타/ 잘못된 주소 기입이 에러의 원인
# 에러가 발생한 역의 도로명 주소 새로 구해서 테스트
kakao_location('경기도 안산시 단원구 선부광장서로 42')

(37.3329390400258, 126.805051368864)

In [ ]:
# 6 작전 인천광역시 계양구 계양대로 지하73 (작전동) 
# 86 청라국제도시 인천광역시 서구 에코로 151 (청라동)
# 102 성신여대입구 서울특별시 성북구 동소문로 지하 102 (동선동4가 1번지)
# 430 성환 충청남도 천안시 서북구 성환읍 성환1로 237-5 (성환리 449-490번지)
# 437 탕정 충청남도 아산시 탕정면 한들물빛6로 11 (매곡리 1606번지)
# 470 덕계 경기도 양주시 덕계로 126 (덕계동 209-5번지)
# 491 야당 경기도 파주시 소리천로 10 (야당동)
# 499 덕소 경기도 남양주시 와부읍 덕소로 56 (덕소리)
# 502 운길산 경기도 남양주시 조안면 운길산로 20 (진중리)
# 504 신원 경기도 양평군 양서면 신원역길 5-4
# 506 아신 경기도 양평군 옥천면 아신역1길 23 
# 508 원덕 경기도 양평군 양평읍 원덕흑천길 94-22
# 511 금릉 경기도 파주시 금릉역로 85 (금촌동)
# 540 사릉 경기도 남양주시 진건읍 진건우회로 63 (사능리 605-3)
# 544 마석 경기도 남양주시 화도읍 마석중앙로 107 (마석우리 222-2)
# 554 춘천 강원도 춘천시 공지로 591 (근화동 190)
# 565 오시리아 부산광역시 기장군 기장읍 동부산관광5로 14
# 574 개운포 울산광역시 남구 산업로 133 (상개동)
# 581 개포동 서울특별시 강남구 개포로 지하 420 (개포동)
# 582 대모산입구 서울특별시 강남구 개포로 지하 522 (일원동 703번지)
# 596 망포 경기도 수원시 영통구 덕영대로 지하 1520 (영통동)
# 601 오목천 경기도 수원시 권선구 삼천병마로 지하 1570 (오목천동 472-3)
# 602 어천 경기도 화성시 매송면 어천역길 8
# 604 사리 경기도 안산시 상록구 충장로 103 (사동)
# 626 대공원 경기도 과천시 대공원대로 지하 50 (과천동)
# 639 원당 경기도 고양시 덕양구 고양대로 1429 (성사동 410-7번지)
# 640 화정 경기도 고양시 덕양구 화정로 60 (화정동 1098번지)
# 653 선부 경기도 안산시 단원구 선부광장서로 42
# 655 원시 경기도 안산시 단원구 산단로 지하 70 (원시동)
# 신논현 서울특별시 강남구 봉은사로 지하 102 (논현동 282번지)

In [128]:
# 전처리 과정에서 누락된 부분 제거
df3 = df[df['도로명주소'].str.contains('부산|울산')]
df3

,지하철역,도로명주소
555,부전,부산광역시 부산진구 부전동 280-76
556,거제해맞이,부산광역시 연제구 거제동 619-1
557,거제,부산광역시 연제구 거제동 619-1
558,동래,부산광역시 동래구 낙민동 130-4
559,안락,부산광역시 동래구 안락동 201-2
560,부산원동,부산시 동래구 안락동 52-2
561,재송,부산광역시 해운대구 재송동 933-1
562,센텀,부산광역시 해운대구 우동 1215-32
563,벡스코,부산광역시 해운대구 우동 1111-2
564,신해운대,부산광역시 해운대구 좌동 138-4


In [129]:
df[df.지하철역.values == '태화강']

,지하철역,도로명주소
575,태화강,울산광역시 남구 산업로 654


In [130]:
df = df[~df['도로명주소'].str.contains('부산|울산')]

In [131]:
df.shape

(635, 2)

In [132]:
df[df.지하철역.values == '태화강']

,지하철역,도로명주소


In [133]:
df.shape

(635, 2)

In [134]:
# 누락된 부분 제거 하는 코드
df = df[~df['도로명주소'].str.contains('부산|울산')]

In [135]:
df.shape

(635, 2)

In [136]:
df.to_csv('local_info.csv',index=False)